# وظيفة 1:

Name1 : محمد خير راتب يدج

ID1 : 021041854

Name2 : علي محمد درويش

ID2 : 020041662

## setup

In [120]:
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
import arabic_reshaper
import arabicstopwords.arabicstopwords as stp
import re
from snowballstemmer import stemmer


from ar_corrector.corrector import Corrector

## Prepare classes

### Preprcessor

#### Task-1:

In [136]:
#-preprocessor
class Preprocessor():
    @staticmethod
    def process(sentence):
        print("The Result is :")
        s = Preprocessor.normDocument(sentence)
        s = Preprocessor.removestopwords(s)
        s = Preprocessor.stemDocument(s)
        # s = Preprocessor.lemmatizDocument(s)
        # s = Preprocessor.correctDocument(s)
        return Preprocessor.tokenizeDocument(s)
    
    @staticmethod
    def tokenizeDocument(sentance):
        tokenizer = WhitespaceTokenizer()
        return tokenizer.tokenize(sentance)
    
    @staticmethod
    def removestopwords(sentence):
        terms=set()
        stopWords= set(stp.stopwords_list())
        for term in Preprocessor.tokenizeDocument(sentence) :
            if term not in stopWords :
                terms.add(term)
        terms = list(terms)
        return " ".join(terms)
    
    @staticmethod
    def stemDocument(sentence):
        ar_stemmer = stemmer("arabic")
        return " ".join([ar_stemmer.stemWord(i) for i in Preprocessor.tokenizeDocument(sentence)])    

    @staticmethod
    def normDocument(sentence):
        text = re.sub("[إأٱآا]", "ا", sentence)
        text = re.sub("ى", "ي", text)
        text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
        text = re.sub("ؤ", "ء", text)
        text = re.sub("ئ", "ء", text)
        text = re.sub("ة", "ه", text)
        return(text)
    
    @staticmethod
    def correctDocument(sentance):
        corr = Corrector() 
        terms = []
        return " ".join([corr.spell_correct(term) for term in Preprocessor.tokenizeDocument(sentance)])
        # for term in Preprocessor.tokenizeDocument(sentance):
        #     terms.append(corr.spell_correct(term))
        # return " ".join([term for term in terms])


In [143]:
d1 = "برنامج تعليمي للغة الإنجليزية ومسار سريع"
d2 = "تعلم الفهرسة الدلالية الكامنة"
d3 = "الكتاب في الفهرسة الدلالية"
d4 = "التقدم في البنية و الفهرسة الدلالية"
d5 = "تحليل تحليل البنية الكامنة"

In [145]:
# Preprocessor.process(".ذهبت, إلي. الغابة")
Preprocessor.process(d1)

The Result is :


['مسار', 'رنامج', 'للغه', 'انجليزيه', 'تعليم', 'سريع']

### TDIM

#### Task-2:

قم بالتعديل على باني الكلاس

IndexModel

بحيث يصبح نوع الفهرس المستخدم  من نمط معطيات جدول, أي

data type of the attribute "_index" is DataFrame

In [17]:
#- Index Model : Term Document Incidence Matrix
class IndexModel:

    def __init__(self, documents_df):
        print("question-2.1")
        self._index = {}
        termdoc = documents_df.to_dict('list')
        for term in termdoc:
            termlist = []
            i = 0
            for term in termdoc:
               i +=1
            termlist.append((termdoc["id"],i))
            self._index[term] = termlist


    def term_incidence_vector(self, term):
        print("question-2.2")
        # اكتب اجابتك هنا
        #return ...





In [ ]:
# { 
#   "term1" : [(d1,5), (d2,1), (d5,7)],
#   "term2" : [(d3,3), (d4,6), (d5,2)],
#   "term3" : [(d1,2), (d2,7), (d3,6)],
#  }

In [32]:
s = IndexModel(documents_df = loadDocuments())
s

question-2.1


### Retriever

In [19]:

class Retriever:
    def __init__(self):
        self._terms_operator = ['&', '|', '~']

    def _merge(self, l1, l2):
        res = []
        i1 = 0; i2 = 0
        while i1 < len(l1) and i2 < len(l2):
            if l1[i1]==l2[i2]:
                res += [l1[i1]]
                i1 += 1; i2 += 1
            elif l1[i1]<l2[i2]:
                i1 += 1
            elif l2[i2]<l1[i1]:
                i2 += 1
        return res

    def boolean_operator_processing(self, bop, prevS, nextS=None, docidlist= None):
        if bop == "&":
            return self._merge(prevS, nextS)
        elif bop=="|" :
            return list(set(prevS+nextS))
        elif bop == "~":
            return [a for a in docidlist if a not in prevS]

    def retrieve(self, query_terms, index_model):
        ret_docs = []
        bitwiseop=""
        result=[]
        has_previous_term=False
        has_not_operation=False
        inc_vec_prev=[]
        inc_vec_next=[]
        for term in query_terms:
            if term not in self._terms_operator:
                if has_not_operation:
                    if has_previous_term:
                        inc_vec_next=self.boolean_operator_processing(bop="~",prevS=index_model.term_postings(term),nextS=inc_vec_next, docidlist=index_model.getdi())
                    else :
                        inc_vec_prev=self.boolean_operator_processing(bop="~",prevS=index_model.term_postings(term),nextS=inc_vec_next, docidlist=index_model.getdi())
                        result=inc_vec_prev
                    has_not_operation=False
                elif has_previous_term:
                    inc_vec_next=index_model.term_postings(term)
                else:
                    inc_vec_prev=index_model.term_postings(term)
                    result= inc_vec_prev
                    has_previous_term=True
            elif term =="~":
                has_not_operation=True
            else:
                bitwiseop=term

            #----------
            if len(inc_vec_next)!= 0  :
                result = self.boolean_operator_processing(bop=bitwiseop,prevS=inc_vec_prev,nextS=inc_vec_next, docidlist=index_model.getdi())
                inc_vec_prev=result
                has_previous_term=True
                inc_vec_next= []

        #-----
        for res in result:
            ret_docs.append({'docno':res, 'score':1})
        ret_docs = pd.DataFrame(ret_docs, columns=['docno', 'score', 'content']).sort_values(by=['score'], ascending=False)
        return ret_docs



### search engine

In [20]:
class SearchEngine:
    def __init__(self, preprocessor, retriever, documents):
        self.preprocessor = preprocessor
        self.retriever = retriever
        self.documents = None
        self.model = None
        self.rebuild(documents)

    def rebuild(self, documents):
        self.documents = documents
        self.documents['ntext'] = self.documents['text'].apply(self.preprocessor.process)
        self.model = IndexModel(self.documents)

    def querying(self, query):
        query_terms = self.preprocessor.process(query)
        docs_res = self.retriever.retrieve(query_terms, self.model)
        if docs_res.shape[0]>0:
            docs_res['content'] = docs_res.apply(
                lambda row: self.documents[self.documents['id']==row.id]['text'].iloc[0], axis = 1)
        return docs_res



## Build and use the IRS

#### Task-3:

انشئ جدول وثائق من مجموعة العبارات التالية


1. برنامج تعليمي للغة الإنجليزية ومسار سريع
2. تعلم الفهرسة الدلالية الكامنة
3. الكتاب في الفهرسة الدلالية
4. التقدم في البنية والفهرسة الدلالية
5. تحليل تحليل البنية الكامنة







In [21]:
d1 = "برنامج تعليمي للغة الإنجليزية ومسار سريع"
d2 = "تعلم الفهرسة الدلالية الكامنة"
d3 = "الكتاب في الفهرسة الدلالية"
d4 = "التقدم في البنية والفهرسة الدلالية"
d5 = "تحليل تحليل البنية الكامنة"

def loadDocuments():
    return pd.DataFrame([
        [0,d1],
        [1,d2],
        [2,d3],
        [3,d4],
        [4,d5],
    ],columns=["id","text"])
df = loadDocuments()
df

,id,text
0,0,برنامج تعليمي للغة الإنجليزية ومسار سريع
1,1,تعلم الفهرسة الدلالية الكامنة
2,2,الكتاب في الفهرسة الدلالية
3,3,التقدم في البنية والفهرسة الدلالية
4,4,تحليل تحليل البنية الكامنة


#### Task-4:

انشئ كيان من نظام محرك البحث

In [22]:
# اكتب اجابتك هنا
#ir_sys = ...

#### Task-5:

اعرض اخر 3 وثائق

واعرض حجم المصفوفة TDIM

In [23]:
# اخر 3 وثائق
# اكتب اجابتك هنا
#...

In [24]:
# حجم مصفوفة وجود الكلمات في الوثائق TDIM
# اكتب اجابتك هنا
#...

#### Task-6:

قم بعرض الوثائق الموافقة للاستعلامات التالية

Query-1 = "التقدم & البنية & ~ تحليل"

In [25]:
# اكتب اجابتك هنا
#...

Query-2 = "الكامنة & ~ تعليمي"

In [26]:
# اكتب اجابتك هنا
#...

Query-3 = "التقدم والبنية بدون تحليل"

In [27]:
# اكتب اجابتك هنا
#...

Query-4 = "تعلم | تحليل & الكامنة"


In [28]:
# اكتب اجابتك هنا
#...